In [11]:
%matplotlib inline

import nengo
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import nengo_dl

# keras uses the global random seeds, so we set those here to
# ensure the example is reproducible
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

from sklearn.datasets import load_iris

In [12]:
iris = load_iris()
X = iris.data
y = iris.target

In [13]:
X.shape

(150, 4)

In [14]:
y.shape

(150,)

In [15]:
set(y)

{0, 1, 2}

In [16]:
with nengo.Network(seed=0) as net:
    # set up some default parameters to match the Keras defaults
    net.config[nengo.Ensemble].gain = nengo.dists.Choice([1])
    net.config[nengo.Ensemble].bias = nengo.dists.Choice([0])
    net.config[nengo.Connection].synapse = None
    net.config[nengo.Connection].transform = nengo_dl.dists.Glorot()

    # input node, same as before
    inp = nengo.Node(output=np.ones(X.shape[1]))

    # add the first dense layer
    hidden = nengo.Ensemble(64, 1, neuron_type=nengo.RectifiedLinear()).neurons
    nengo.Connection(inp, hidden)
    hidden2 = nengo.Ensemble(64, 1, neuron_type=nengo.RectifiedLinear()).neurons
    nengo.Connection(hidden, hidden2)

    # add the linear output layer (using nengo.Node since there is
    # no nonlinearity)
    out = nengo.Node(size_in=len(set(y)))
    nengo.Connection(hidden2, out)

    # add a probe to collect output
    out_p = nengo.Probe(out)

In [17]:
from sklearn.model_selection import train_test_split
X_snn = X[:, None, :]
y_snn = y[:, None, None]
X_train, X_test, y_train, y_test = train_test_split(X_snn, y_snn, test_size=0.2, random_state=0)

In [18]:
X_train.shape

(120, 1, 4)

In [19]:
y_train.shape

(120, 1, 1)

In [20]:
with net:
    nengo_dl.configure_settings(stateful=False, use_loop=False)

with nengo_dl.Simulator(net, minibatch_size=6) as sim:
    sim.compile(optimizer=tf.optimizers.Adam(),
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"])
    sim.fit(X_train, y_train, epochs=10)

    print("Test accuracy:", sim.evaluate(
        X_test, y_test, verbose=0)["probe_accuracy"])
    
    sim.save_params("./iris_snn")

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Train on 120 samples


InternalError: GPU sync failed

In [21]:
from sklearn.metrics import f1_score, accuracy_score
with nengo_dl.Simulator(net, minibatch_size=6) as sim2:
    sim2.load_params("./iris_snn")
    predictions = sim2.predict(X_test)[out_p]
    y_pred = []
    for p in predictions:
        y_pred.append(np.argmax(p[0]))
    sim2.compile(optimizer=tf.optimizers.Adam(),
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=["accuracy"])
    print("Test loss: ", sim2.evaluate(X_test, y_test, verbose=0)['loss'])
    print("Test accuracy: ", accuracy_score(y_test.flatten(), y_pred))
    print("Test F1 Score: ", f1_score(y_test.flatten(), y_pred, average='micro'))

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               


InternalError: GPU sync failed

## Keras layers

In [98]:
with nengo.Network(seed=seed) as net:
    # input node, same as before
    inp = nengo.Node(output=np.ones(X.shape[1]))

    # add the Dense layers, as in the Keras model
    hidden = nengo_dl.Layer(
        tf.keras.layers.Dense(units=64, activation=tf.nn.relu))(inp)
    hidden2 = nengo_dl.Layer(
        tf.keras.layers.Dense(units=64, activation=tf.nn.relu))(hidden)
    out = nengo_dl.Layer(
        tf.keras.layers.Dense(units=len(set(y))))(hidden2)

    # add a probe to collect output
    out_p = nengo.Probe(out)

In [56]:
X_train.shape

(120, 1, 4)

In [57]:
y_train.shape

(120, 1, 1)

In [100]:
with net:
    nengo_dl.configure_settings(stateful=False, use_loop=False)

with nengo_dl.Simulator(net, minibatch_size=6) as sim:
    sim.compile(optimizer=tf.optimizers.Adam(),
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"])
    sim.fit(X_train, y_train, epochs=10)

    print("Test accuracy:", sim.evaluate(
        X_test, y_test, verbose=0)["probe_accuracy"])

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Train on 120 samples
Epoch 1/10
120/120 [==============================] - 0s 933us/sample - loss: 1.0128 - probe_loss: 1.0128 - probe_accuracy: 0.5750
Epoch 2/10
120/120 [==============================] - 0s 450us/sample - loss: 0.8427 - probe_loss: 0.8427 - probe_accuracy: 0.6583
Epoch 3/10
120/120 [==============================] - 0s 433us/sample - loss: 0.6728 - probe_loss: 0.6728 - probe_accuracy: 0.7250
Epoch 4/10
120/120 [==============================] - 0s 458us/sample - loss: 0.5555 - probe_loss: 0.5555 - probe_accuracy: 0.7750
Epoch 5/10
120/120 [==============================] - 0s 433us/sample - loss: 0.4800 - probe_loss: 0.4800 - probe_accuracy: 0.8667
Epoch 6/10
120/120 [==============================] - 0s 425us/sample - loss: 0.4119 - probe_loss: 0.4119 - probe_accuracy: 0.9083
Epoch 7/10
120/120 [=============

In [101]:
with nengo_dl.Simulator(net, minibatch_size=6) as sim:
    predictions = sim.predict(X_test)[out_p]
    y_pred = []
    for p in predictions:
        y_pred.append(p[0])
y_pred

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               


[array([ 0.3828    , -0.10269964,  0.1600502 ], dtype=float32),
 array([ 0.4032839 , -0.16633818,  0.38782042], dtype=float32),
 array([ 0.10945285, -0.00089379,  0.07321441], dtype=float32),
 array([ 0.49845922, -0.09372929,  0.3818662 ], dtype=float32),
 array([ 0.13050717, -0.02711663,  0.11718857], dtype=float32),
 array([ 0.4298821 , -0.01159807,  0.10131407], dtype=float32),
 array([ 0.12207904, -0.05359422,  0.12546197], dtype=float32),
 array([ 0.41004848, -0.09425357,  0.25363475], dtype=float32),
 array([ 0.4382305 , -0.13365176,  0.34510392], dtype=float32),
 array([ 0.3618262 , -0.11051926,  0.24229154], dtype=float32),
 array([ 0.41096488, -0.01255447,  0.28590828], dtype=float32),
 array([ 0.3774116 , -0.05764061,  0.17933172], dtype=float32),
 array([ 0.39414456, -0.01771963,  0.22566038], dtype=float32),
 array([ 0.41148368, -0.13054562,  0.29951066], dtype=float32),
 array([ 0.39243472, -0.02714005,  0.20145038], dtype=float32),
 array([0.10534459, 0.01297808, 0.066730